# Data and Libraries loading  

In [ ]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# will be used to load MATLAB mat datafile format
from scipy.io import loadmat

# library written for this exercise providing additional functions for assignment submission, and others
import utils

### Loading the data 

In [2]:
#  training data stored in arrays X, y
data = loadmat(os.path.join('Data', 'ex4data1.mat'))
X, y = data['X'], data['y'].ravel()

# set the zero digit to 0, rather than its mapped 10 in this dataset
# This is an artifact due to the fact that this dataset was used in 
# MATLAB where there is no index 0
y[y == 10] = 0

# Number of training examples
m = y.size

## Neural network

### Model representation
![](Figures/neural_network.png)

### Setting up the neural network 

##### input layer size : 
The first layer (input) consists of $400$ neurons(without counting the bias). 
Because given a $\mathbb{R}^{20*20}$ by image, we have then a matrix of $\mathbb{R}^{20*20}$ which we can then turn into a vector of dimension $\mathbb{R}^{400}$.

In such way every image in our dataset X can be turned into a vector of dimension $\mathbb{R}^{400}$. 
Finally our dataset of $5000$ images will be a $\mathbb{R}^{5000*400}$ matrix. 

`m` : being the number of training examples 
every row of the matrix `X` : represents a training example 


$$ X = \begin{bmatrix} - \left(x^{(1)} \right)^T - \\
- \left(x^{(2)} \right)^T - \\
\vdots \\
- \left(x^{(m)} \right)^T - \\
\end{bmatrix}
$$


We will have a matrix later that represents the y expected outcome 

In [3]:
input_layer_size  = 400  

### Second layer 

In [4]:
# 25 hidden units (neurons)
hidden_layer_size = 25  

### Third layer (input layer): 

In [5]:
# 10 labels, from 0 to 9 (this is )
num_labels = 10          

### Weights Loading 

In [3]:
# Load the weights into variables Theta1 and Theta2
weights = loadmat(os.path.join('Data', 'ex4weights.mat'))

# Theta1 has size 25 x 401
# Theta2 has size 10 x 26
Theta1, Theta2 = weights['Theta1'], weights['Theta2']

# swap first and last columns of Theta2, due to legacy from MATLAB indexing, 
# since the weight file ex3weights.mat was saved based on MATLAB indexing
Theta2 = np.roll(Theta2, 1, axis=0)

# Unroll parameters 
nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])

# Implementing the cost function 

#### Without regularization : 

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^{m}\sum_{k=1}^{K} \left[ - y_k^{(i)} \log \left( \left( h_\theta \left( x^{(i)} \right) \right)_k \right) - \left( 1 - y_k^{(i)} \right) \log \left( 1 - \left( h_\theta \left( x^{(i)} \right) \right)_k \right) \right]$$


##### With regularization : 

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^{m}\sum_{k=1}^{K} \left[ - y_k^{(i)} \log \left( \left( h_\theta \left( x^{(i)} \right) \right)_k \right) - \left( 1 - y_k^{(i)} \right) \log \left( 1 - \left( h_\theta \left( x^{(i)} \right) \right)_k \right) \right] + \frac{\lambda}{2 m} \left[ \sum_{j=1}^{25} \sum_{k=1}^{400} \left( \Theta_{j,k}^{(1)} \right)^2 + \sum_{j=1}^{10} \sum_{k=1}^{25} \left( \Theta_{j,k}^{(2)} \right)^2 \right] $$


## Feedforward Propagation 
****
To implement the cost function we will have first to implement the Feedforward propagation function 

Given a dataset $X$, parameters $\theta _{1}$ and $ \theta _{2}$. We will be able to compute 
$h(x)= a_{3}$ for the a whole dataset or for just one training example 

In [1]:
def feedForwardProp(Theta1, Theta2 , X): 
    if X.ndim == 1:
        X = X[None]  # promote to 2-dimensions
    
    # useful variables
    m = X.shape[0]
    num_labels = Theta2.shape[0]
    
    # We are going to add a column of one's to X for bias
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    
    z_2 = Theta1 @  (X.T)
    a_2 = utils.sigmoid(z_2)
    
    # Again here are going to add a column of one's to X for bias 
    a_2 = np.concatenate([np.ones((1, m)), a_2], axis=0)
    
    z_3 = Theta2 @ a_2
    a_3 = utils.sigmoid(z_3)
    
    return (a_3,z_3,a_2,z_2,X)  

## Cost function and Backpropagation  

In [7]:
def nnCostFunction(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lambda_=0.0):
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))
    
    
    # Calculating cost function - non regularized  
    
    m = y.size
    
    # we will transform the y vector which contains the expected outputs from the 
    # training examples in a matrix of dimensions K x m 
    # each column in this matrix represents expected outputs for each unit from layer 3
    
    y_transformed = np.zeros((m,num_labels))
    
    for idx in range(m): 
        y_transformed[idx,y[idx]] = 1 
    
    
    '''Cost function unregularized vectorized version'''
    a_3 = feedForwardProp(Theta1, Theta2, X)[0]
    
    # For calculating the cost function J in this case without using a for loop 
    # we will use a pair-wise matrix multiplication 
    J = np.sum((-(y_transformed.T)*np.log(a_3) - (1-y_transformed.T)*np.log(1-a_3))) / m 
    
    
    '''Cost function regularized vectorized version'''
    sum_1 = np.sum(Theta1[:,1:]*Theta1[:,1:])
    sum_2 = np.sum(Theta2[:,1:]*Theta2[:,1:])
    
    final_sum = sum_1 + sum_2 
    J += ((final_sum*(lambda_))/(2*m))
        
    
    
    
    '''Calculating the gradient using backpropagation'''
    
    delta_1 = 0
    delta_2 = 0
        
    for t in range(m):
        curr_y = np.array(y_transformed[t, :])
        a_3 = feedForwardProp(Theta1, Theta2, X[t,:])[0]
        layer3_error = a_3.T - curr_y
        layer3_error = layer3_error.T
        
        
        a_2 = feedForwardProp(Theta1, Theta2, X[t,:])[2]
        a_2_term = np.multiply(a_2 ,(1-a_2))
    
        layer2_error = ((Theta2.T)@ layer3_error) * a_2_term
        
        
        delta_2 += layer3_error@ (a_2.T) 
        
        a_1 = feedForwardProp(Theta1, Theta2, X[t,:])[4]
        layer2_error = layer2_error[1:]
        delta_1 += layer2_error@ (a_1)
        
        
    
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    
    Theta1_grad[:,0] = delta_1[:,0]/m 
    Theta2_grad[:,0] = delta_2[:,0]/m 
    
    Theta1_grad[:,1:] = delta_1[:,1:]/m + (lambda_/m) * Theta1[:,1:]
    Theta2_grad[:,1:] = delta_2[:,1:]/m + (lambda_/m) * Theta2[:,1:]
    
    
    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])

    return J, grad
    

# Training the neural network 


## Random initialization
When training neural networks, it is important to randomly initialize the parameters for symmetry breaking. One effective strategy for random initialization is to randomly select values for $\Theta^{(l)}$ uniformly in the range $[-\epsilon_{init}, \epsilon_{init}]$. You should use $\epsilon_{init} = 0.12$. This range of values ensures that the parameters are kept small and makes the learning more efficient.

<div class="alert alert-box alert-warning">
One effective strategy for choosing $\epsilon_{init}$ is to base it on the number of units in the network. A good choice of $\epsilon_{init}$ is $\epsilon_{init} = \frac{\sqrt{6}}{\sqrt{L_{in} + L_{out}}}$ where $L_{in} = s_l$ and $L_{out} = s_{l+1}$ are the number of units in the layers adjacent to $\Theta^{l}$.
</div>

In [2]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12): 
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

    return W

## Learning the parameters 

In [ ]:
#  After you have completed the assignment, change the maxiter to a larger
#  value to see how more training helps.
options= {'maxiter': 100}

#  You should also try different values of lambda
lambda_ = 1

# Create "short hand" for the cost function to be minimized
costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, X, y, lambda_)

# Now, costFunction is a function that takes in only one argument
# (the neural network parameters)
res = optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='TNC',
                        options=options)

# get the solution of the optimization
nn_params = res.x
        
# Obtain Theta1 and Theta2 back from nn_params
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))